In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import glob
import re
import json
import os.path
import time
import re


In [2]:
data_set = []

In [3]:
paths = glob.glob("/home/parth/information_retrieval/Data/AP89_DATA/AP_DATA/ap89_collection/*")

In [4]:
x = 0
doc_total = 0
for path in paths:
    try:
        fnames = path.split("/")[-1]
        f = open(path,"r", encoding='iso-8859-1')
        text = f.read()
        doc_total = doc_total + text.count('<DOC>')
        for m in zip(re.finditer('<DOC>', text), re.finditer('</DOC>', text)):
            docdata = text[m[0].start():m[1].end()]
            file_data = {}
            for m in zip(re.finditer('<DOCNO>', docdata), re.finditer('</DOCNO>', docdata)):
                doc_no = docdata[m[0].end() + 1 : m[1].start()]
                file_data['DOCNO'] = doc_no

            file_data['TEXT'] = ""
            for m in zip(re.finditer('<TEXT>', docdata), re.finditer('</TEXT>', docdata)):
                file_data['TEXT'] = file_data['TEXT'] + docdata[m[0].end() + 1: m[1].start()]
#             print(file_data)
            data_set.append(file_data)
    except:
        print(path + " error in file generation")

In [52]:
stop_files = open("stoplist.txt", "r")

In [53]:
stopwords = [snowball_stemmer.stem(stopword) for stopword in stop_files.read().split("\n")]

In [5]:
# numbers_checker = re.compile(r"[0-9]+\.[0-9]+|[0-9]+")
# numbers_checker = re.compile(r"([0-9+\.]+$)")
# patern = re.compile(r'?\d+(?:,\d*)?')
# pattern.match("123,12").start()
# "123.134".count(r"[0-9]+.[0-9]+")
# numbers_checker_comma = re.compile(r"([0-9]+,[0-9]+\.[0-9]+|[0-9]+,[0-9]+)")
# numbers_checker = re.compile(r"(\d{1,0})")
# len(re.findall(pattern, "12312"))
# re.findall(numbers_checker, "5110.")

numbers_checker = re.compile(r"(^[\d][\d+\.]+$)")
numbers_checker_comma = re.compile(r"^[\d][\d+\.\,]+$")

In [56]:
vocab = {}
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[A-Za-z0-9]+') // 

from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")

docid = 0
docmap = {}
reversedocmap = {}
vocabmap = {}
index = 0
reverse_vocab = {}

file  = open("tokenisation", "w")
start = time.time()
for doc in data_set:
    pos = 0
    docmap[doc['DOCNO']] = docid
    reversedocmap[docid] = doc['DOCNO']
    for token in doc['TEXT'].split():
        token = token.lower()
        if(len(re.findall(numbers_checker, token)) == 1):
            token = token
        elif(len(re.findall(numbers_checker_comma,token)) == 1):
            token = token.replace(",","")
        else:
            # token = snowball_stemmer.stem(re.sub(r"[\.]|[,]|[']|[\"]|[_]|[`]", "", token))
            token = tokenizer.tokenize(token)
            if(len(token) > 0):
                token = token[0]
            else:
                token = ""
            token = snowball_stemmer.stem(token)

        if token not in stopwords :
            if token not in vocab.keys():
                vocab[token] = 0 
                vocabmap[token] = index
                index = index + 1
                reverse_vocab[index] = token

            vocab[token] = vocab[token] + 1
            pos = pos + 1
    #         print("(" + str(vocabmap[token]) + " " + str(docid) + " " + str(pos) + ") " , end=', ' )
            file.write("(" + str(vocabmap[token]) + " " + str(docid) + " " + str(pos) + ") " )

    docid = docid + 1

end = time.time()
print(end - start)

649.0925583839417


In [62]:
vocab['northwestern']

805

In [60]:
vocabmap['maximum']

3897

In [61]:
reverse_vocab[295]

'northwestern'

In [63]:
cnt = 0
for key in vocab.keys():
    cnt = cnt + vocab[key]

In [64]:
cnt 

20242236

In [66]:
with open('vocab.json', 'w') as outfile:
    json.dump(vocab, outfile)
    
with open('docmap.json', 'w') as outfile:
    json.dump(docmap, outfile)
    
with open('reversedocmap.json', 'w') as outfile:
    json.dump(reversedocmap, outfile)
    
with open('vocabmap.json', 'w') as outfile:
    json.dump(vocabmap, outfile)

    
with open('reverse_vocab.json', 'w') as outfile:
    json.dump(vocabmap, outfile)


In [142]:
txt = "0 144 105 84068 1 275 84229 1 336 84012 1 23 84490 1 63 84491 3 25 94 132 84631 1 157 84237 1 177 84242 1 8 84500 1 18 84503 2 62 83 84257 1 137 84258 1 137 84059 1 150 84004 2 80 339 84262 1 274 84572 1 52 84268 1 119 84278 1 213 84279 2 49 112 84280 2 172 261 84025 1 250 84191 1 117 84030 1 265 84031 1 304 84036 2 150 246 84662 2 26 79 84038 1 61 84551 3 8 21 84 84040 1 187 84555 1 121 84045 1 46 84047 5 53 72 162 218 246 84560 1 102 84051 1 195 84564 1 115 84054 1 168 84281 3 27 89 150 84056 1 168 84324 1 355 84315 1 136 84316 1 82 84666 2 40 56 84318 1 28 84578 1 59 84310 1 193 84438 1 219 84586 1 250 84075 1 308 84079 1 146 84592 5 2 29 145 209 222 84627 2 172 186 84286 2 318 338 84087 2 337 342 84600 1 597 84090 1 199 84092 2 11 16 84605 1 47 84607 1 46 84358 1 162 84615 1 129 84247 1 119 84370 2 63 67 84371 1 249 84116 1 79 84630 1 61 84119 3 23 48 81 84633 1 206 84122 1 72 84381 1 44 84126 1 44 84385 1 241 84387 1 52 84645 1 50 84648 2 17 69 84653 2 31 194 84655 3 3 50 144 84168 2 103 232 84406 1 160 84151 1 359 84154 1 400 84156 1 291 84157 1 312 84674 1 60 84163 1 103 84420 2 14 88 84214 1 152 84424 1 165 84172 2 385 418 84176 1 63 84434 2 146 241 84435 1 90 84182 2 141 343 84188 1 328 84190 1 437 84447 1 218 84192 1 214 84454 2 475 479 84455 1 204 84462 1 267 84209 1 160 84210 1 90 84212 1 47 84470 2 13 190 84217 1 140 84221 1 42 "

In [143]:
with open('test', 'w') as outfile:
    outfile.write(txt)

In [55]:
file = open('/home/parth/information_retrieval/cs6200-assignment/assignment-2/testfiles/test_file_1', 'r')
file.seek(14)
txt = file.readline()
print(len(txt))
txt = txt.replace("\n", "").split(" ")
print(txt)
txt = [ int(x) for x in txt]
print(txt)


14
['1', '2', '1', '0', '2', '2', '3']
[1, 2, 1, 0, 2, 2, 3]


In [50]:
# file.seek(49279)

In [56]:
txt

[1, 2, 1, 0, 2, 2, 3]

In [57]:
doc_pointer = 3
doc_f = int(txt[2])
for x in range(0,doc_f):
    print("document_id:",  txt[doc_pointer])
    print("document frequency:",  txt[doc_pointer + 1])
    tf = int(txt[doc_pointer + 1])
    tf_pointer = doc_pointer + 1
    for i in range(1, tf+1):
        print("posistion  :", txt[tf_pointer + i])
    doc_pointer = 1 + (doc_pointer + 1) + int(txt[(doc_pointer + 1)]) 
    print("")

document_id: 0
document frequency: 2
posistion  : 2
posistion  : 3

